## Test case LVV-T2724: 

### Specification: 
This test covers three requirements relating to high-volume queries on Qserv

####  Requirement ID: DMS-REQ-0354 (Priority: 1b) Result latency for high-volume complex queries<br>
Specification: Complex high-volume queries -– queries that involve full-sky spatial and temporal correlations -- shall be answered in less than hvComplexQueryTime. 
hvComplexQueryTime:  Maximum time allowed for retrieving results of a high-volume complex query. == 12 hours

#### Requirements ID: DMS-REQ-0361 (Priority: 1b)
Specification: The system shall support hvQueryUsers simultaneous high-volume queries running at any given time
hvQueryUsers: Minimum number of simultaneous users performing high volume queries == 50 

### Discussion:
We will use the DP1 catalog fr now. This is a query on Qserv, not the Butler

### Data 
DP1 full scan is teh best we can do 2025-07-22
Eventually erify latency against a full scale, e.g DR1 sized Object catalog


In [1]:
from lsst.rsp import get_tap_service
import time

In [ ]:
# Requirements specs

# Maximum time allowed for retrieving results of a high-volume query of the Object table. Hours
hvComplexQueryTime = 12 

# Minimum number of simultaneous users performing high volume queries.
hvQueryUsers = 50 

In [2]:
service = get_tap_service("tap")
assert service is not None

In [3]:
# Calculate size in gigabytes (GB)
def get_size_in_gb(df): 
    size = df.memory_usage(deep=True).sum()
    size /= 1024 ** 3
    return size

In [5]:
# High-volume complex query involving  full-sky spatial and temporal correlations and use of some internal functions 
# query = """SELECT *
# FROM dp1.ForcedSourceOnDiaObject as fsodo
# JOIN dp1.Visit as vis ON vis.visit = fsodo.visit 
# ORDER BY diaObjectId ASC


#     scisql_nanojanskyToAbMag(g_sersicFlux) AS g_sersicMag,
#     scisql_nanojanskyToAbMagSigma(g_sersicFlux, g_sersicFluxErr) AS g_sersicMagErr,
#     scisql_nanojanskyToAbMag(r_sersicFlux) AS r_sersicMag,
#     scisql_nanojanskyToAbMagSigma(r_sersicFlux, r_sersicFluxErr) AS r_sersicMagErr

# FROM dp1.Object
# """

query = """
        SELECT *
        FROM dp1.DiaObject AS diao
        JOIN dp1.ForcedSourceOnDiaObject AS fsodo ON fsodo.diaObjectId = diao.diaObjectId
        JOIN dp1.Visit AS v ON v.visit = fsodo.visit
        
print(query)

SELECT coord_ra, coord_dec
       FROM dp1.Object 
       WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),CIRCLE('ICRS', 53.01, -28.35, 0.016666666666666666))=1



In [6]:
job = service.submit_job(query)

In [7]:
start_time = time.time()
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
if job.phase == 'ERROR':
    job.raise_if_error()
assert job.phase == 'COMPLETED'
results = job.fetch_result()
end_time = time.time()

Job phase is COMPLETED


In [8]:
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

Execution time: 1.2185323238372803


In [9]:
df = results.to_table().to_pandas()
df.columns

Index(['coord_ra', 'coord_dec'], dtype='object')

In [10]:
job.delete()
del query, results

In [11]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   coord_ra   223 non-null    float64
 1   coord_dec  223 non-null    float64
dtypes: float64(2)
memory usage: 3.6 KB


In [12]:
size = get_size_in_gb(df)
print(size)

DataFrame size: 0.0000 GB
3.4458935260772705e-06


In [13]:
# time_query = 

In [14]:

assert time_query <= hvObjectQueryTime

In [15]:
# # Test code -- fails on usdf but not cloud?
# from lsst.rsp import get_tap_service
# service = get_tap_service("tap")

# query = """SELECT TOP 100 * FROM dp1.Object 
# """
# job = service.submit_job(query)
# job.run()
# job.wait(phases=['COMPLETED', 'ERROR'])
# print('Job phase is', job.phase)
# if job.phase == 'ERROR':
#     job.raise_if_error()
# assert job.phase == 'COMPLETED'
# results = job.fetch_result().to_pandas()

Job phase is ERROR


DALQueryError: Query Error: ClassCastException:null